In [16]:
 !pip install geopy --user
 !pip install folium --user

In [17]:
import pandas as pd
import numpy as np
import math
import geopy.distance
import scipy
import folium
from folium import plugins

ModuleNotFoundError: No module named 'geopy'

In [4]:
folder = "/workspace/vrategov/00.Data/"
topo = "sofia_topo.csv"
industry = "industrial_pollution_latlon.csv"
meteo = "weather_lbsf_20161101-20161130_IP_train.csv"
stations = "stations.csv"

df_topo = pd.DataFrame(pd.read_csv(folder+topo))
df_ind = pd.DataFrame(pd.read_csv(folder+industry))
df_meteo = pd.DataFrame(pd.read_csv(folder+meteo))
df_stations = pd.DataFrame(pd.read_csv(folder+stations,sep=";"))

In [5]:
df_topo.head()

,Lat,Lon,Elev
0,42.62,23.220000,1184.0
1,42.62,23.233571,1333.0
2,42.62,23.247143,1505.0
3,42.62,23.260714,1586.0
4,42.62,23.274286,1533.0


In [6]:
df_ind.head()

,Lat,Lon,m,t/y
0,42.737961,23.241339,8.0,0.38
1,42.662781,23.388806,15.0,0.03
2,42.662908,23.388686,15.0,0.20
3,42.662972,23.388631,15.0,0.96
4,42.663089,23.389250,15.0,1.58


In [7]:
df_meteo.head()

,year,Month,day,TASMAX,TASAVG,TASMIN,DPMAX,DPAVG,DPMIN,RHMAX,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016,11,1,12.7788,6.6672,0.0000,2.2224,-1.1112,-3.3336,87,...,24.14010,12.07005,0.0,1026.753044,1024.890530,1023.028016,-9999,0.00,-9999,5.471756
1,2016,11,2,15.5568,6.6672,-1.6668,2.7780,0.5556,-2.7780,100,...,11.26538,5.63269,0.0,1026.414405,1020.826864,1015.239322,-9999,0.00,-9999,8.046700
2,2016,11,3,13.3344,8.3340,3.3336,7.2228,3.3336,-1.1112,100,...,28.96812,14.48406,0.0,1023.366655,1019.133669,1014.900683,-9999,5.08,-9999,7.563898
3,2016,11,4,10.5564,6.1116,1.6668,6.1116,3.3336,1.1112,100,...,28.96812,14.48406,0.0,1025.398488,1023.197336,1020.996183,-9999,0.00,-9999,9.816974
4,2016,11,5,15.0012,8.8896,2.7780,10.0008,6.6672,2.7780,100,...,14.48406,7.24203,0.0,1024.043933,1020.657544,1017.271155,-9999,0.00,-9999,9.977908


In [8]:
df_stations

,AirQualityStationEoICode,CommonName,Longitude,Latitude
0,BG0040A,Nadezhda,23.310972,42.732292
1,BG0050A,Hipodruma,23.296786,42.680558
2,BG0052A,Druzhba,23.400164,42.666508
3,BG0073A,IAOS/Pavlovo,23.268403,42.669797


In [9]:
def concentration(y,q,u,h,sigma_y,sigma_z,z = 0):
    """Estimate the concentration of PM10 in a point in space.
    
    Keyword arguments:
    y       -- meters crosswind from the emission plume centerline, assumed to be equal to x in our model
    z       -- position in the z direction, default set to 0 to equal ground level(where are the people)
    q       -- stack emissions (g/s)
    u       -- wind speed (m/s)
    h       -- pollutant release height
    sigma_y -- horizontal standard deviation of the emission distribution, in m 
    sigma_z -- vertical standard deviation of the emission distribution, in m 
    """    
    
    c = (q/2*np.pi*u*sigma_y*sigma_z) * (np.exp((-y**x)/(2*sigma_y**2))) * (np.exp((-(z-h)**2)/(2*sigma_z**2))) * (np.exp((-(z+h)**2)/(2*sigma_z**2)))
    
    return(c)

In [10]:
# transform some variables in correct units

df_ind["t/y"] = df_ind["t/y"] * 1000000/(365*24*60*60) # convert the debit to g/s
df_meteo["sfcWindAVG"] = df_meteo["sfcWindAVG"] * 1000/3600 # convert wind speed in m/s

In [11]:
topo_polution = pd.DataFrame({'Date': [], 'Lat': [], 'Lon': [], 'Ind_P10': []})

for i in range(0,df_meteo.shape[0]):
    for j in range(0,df_topo.shape[0]):
        c = 0
        for k in range(0,df_ind.shape[0]):
            a = (df_topo["Lat"][j], df_topo["Lon"][j])
            b = (df_ind["Lat"][k],df_ind["Lon"][k])

            x = geopy.distance.distance(a, b).km #distance in kilometers

            if x < 1:
                sigma_y = 50.5 * x**(0.894)
                sigma_z = 22.8 * x**(0.675-1.3)
            else:
                sigma_y = 50.5 * x**(0.894)
                sigma_z = 55.4 * x**(0.305-34.0)

            c += concentration(y = x ,
                              q = df_ind["t/y"][k],
                              u = df_meteo["sfcWindAVG"][i],
                              h = df_ind["m"][k],
                              sigma_y = sigma_y,
                              sigma_z = sigma_z)
        topo_polution = topo_polution.append({'Date': i, 'Lat': df_topo["Lat"][j], 'Lon': df_topo["Lon"][j], 'Ind_P10': c}, ignore_index=True)
    print('Calculations for day {} are ready.'.format(i))

NameError: name 'geopy' is not defined

In [12]:
topo_polution.head()

,Date,Lat,Lon,Ind_P10


In [13]:
topo_polution.describe()

,Date,Lat,Lon,Ind_P10
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


In [1]:
# functions taken from http://python.hydrology-amsterdam.nl/moduledoc/index.html
def ea_calc(airtemp= scipy.array([]),\
            rh= scipy.array([])):
    '''
    Function to calculate actual vapour pressure from relative humidity:
    
    .. math::    
        e_a = \\frac{rh \\cdot e_s}{100}
        
    where es is the saturated vapour pressure at temperature T.

    Parameters:
        - airtemp: array of measured air temperatures [Celsius].
        - rh: Relative humidity [%].

    Returns:
        - ea: array of actual vapour pressure [Pa].

    Examples
    --------
    
        >>> ea_calc(25,60)
        1900.0946514729308

    '''
    
    # Test input array/value
    #airtemp,rh = _arraytest(airtemp, rh)

    # Calculate saturation vapour pressures
    es = es_calc(airtemp)
    # Calculate actual vapour pressure
    eact = rh / 100.0 * es
    return eact # in Pa

def es_calc(airtemp= scipy.array([])):
    '''
    Function to calculate saturated vapour pressure from temperature.

    For T<0 C the saturation vapour pressure equation for ice is used
    accoring to Goff and Gratch (1946), whereas for T>=0 C that of
    Goff (1957) is used.
    
    Parameters:
        - airtemp : (data-type) measured air temperature [Celsius].
        
    Returns:
        - es : (data-type) saturated vapour pressure [Pa].

    References
    ----------
    
    - Goff, J.A.,and S. Gratch, Low-pressure properties of water from -160 \
    to 212 F. Transactions of the American society of heating and \
    ventilating engineers, p. 95-122, presented at the 52nd annual \
    meeting of the American society of \
    heating and ventilating engineers, New York, 1946.
    - Goff, J. A. Saturation pressure of water on the new Kelvin \
    temperature scale, Transactions of the American \
    society of heating and ventilating engineers, pp 347-354, \
    presented at the semi-annual meeting of the American \
    society of heating and ventilating engineers, Murray Bay, \
    Quebec. Canada, 1957.

    Examples
    --------    
        >>> es_calc(30.0)
        4242.725994656632
        >>> x = [20, 25]
        >>> es_calc(x)
        array([ 2337.08019792,  3166.82441912])
    
    '''

    # Test input array/value
    #airtemp = _arraytest(airtemp)

    # Determine length of array
    n = scipy.size(airtemp)
    # Check if we have a single (array) value or an array
    if n < 2:
        # Calculate saturated vapour pressures, distinguish between water/ice
        if airtemp < 0:
            # Calculate saturation vapour pressure for ice
            log_pi = - 9.09718 * (273.16 / (airtemp + 273.15) - 1.0) \
                     - 3.56654 * math.log10(273.16 / (airtemp + 273.15)) \
                     + 0.876793 * (1.0 - (airtemp + 273.15) / 273.16) \
                     + math.log10(6.1071)
            es = math.pow(10, log_pi)   
        else:
            # Calculate saturation vapour pressure for water
            log_pw = 10.79574 * (1.0 - 273.16 / (airtemp + 273.15)) \
                     - 5.02800 * math.log10((airtemp + 273.15) / 273.16) \
                     + 1.50475E-4 * (1 - math.pow(10, (-8.2969 * ((airtemp +\
                     273.15) / 273.16 - 1.0)))) + 0.42873E-3 * \
                     (math.pow(10, (+4.76955 * (1.0 - 273.16\
                     / (airtemp + 273.15)))) - 1) + 0.78614
            es = math.pow(10, log_pw)
    else:   # Dealing with an array     
        # Initiate the output array
        es = scipy.zeros(n)
        # Calculate saturated vapour pressures, distinguish between water/ice
        for i in range(0, n):              
            if airtemp[i] < 0:
                # Saturation vapour pressure equation for ice
                log_pi = - 9.09718 * (273.16 / (airtemp[i] + 273.15) - 1.0) \
                         - 3.56654 * math.log10(273.16 / (airtemp[i] + 273.15)) \
                         + 0.876793 * (1.0 - (airtemp[i] + 273.15) / 273.16) \
                         + math.log10(6.1071)
                es[i] = math.pow(10, log_pi)
            else:
                # Calculate saturation vapour pressure for water  
                log_pw = 10.79574 * (1.0 - 273.16 / (airtemp[i] + 273.15)) \
                         - 5.02800 * math.log10((airtemp[i] + 273.15) / 273.16) \
                         + 1.50475E-4 * (1 - math.pow(10, (-8.2969\
                         * ((airtemp[i] + 273.15) / 273.16 - 1.0)))) + 0.42873E-3\
                         * (math.pow(10, (+4.76955 * (1.0 - 273.16\
                         / (airtemp[i] + 273.15)))) - 1) + 0.78614
                es[i] = pow(10, log_pw)
    # Convert from hPa to Pa
    es = es * 100.0
    return es # in Pa
def rho_calc(airtemp= scipy.array([]),\
             rh= scipy.array([]),\
             airpress= scipy.array([])):
    '''
    Function to calculate the density of air, rho, from air
    temperatures, relative humidity and air pressure.

    .. math::    
        \\rho = 1.201 \\cdot \\frac{290.0 \\cdot (p - 0.378 \\cdot e_a)}{1000 \\cdot (T + 273.15)} / 100
        
    Parameters:
        - airtemp: (array of) air temperature data [Celsius].
        - rh: (array of) relative humidity data [%].
        - airpress: (array of) air pressure data [Pa].
        
    Returns:
        - rho: (array of) air density data [kg m-3].
        
    Examples
    --------
    
        >>> t = [10, 20, 30]
        >>> rh = [10, 20, 30]
        >>> airpress = [100000, 101000, 102000]
        >>> rho_calc(t,rh,airpress)
        array([ 1.22948419,  1.19787662,  1.16635358])
        >>> rho_calc(10,50,101300)
        1.2431927125520903
        
    '''

    # Test input array/value    
    #airtemp,rh,airpress = _arraytest(airtemp,rh,airpress)
    
    # Calculate actual vapour pressure
    eact = ea_calc(airtemp, rh)
    # Calculate density of air rho
    rho = 1.201 * (290.0 * (airpress - 0.378 * eact)) \
             / (1000.0 * (airtemp + 273.15)) / 100.0
    return rho # in kg/m3

NameError: name 'scipy' is not defined

In [14]:
air_density = pd.DataFrame(rho_calc(df_meteo["TASAVG"], df_meteo["RHAVG"], df_meteo["PSLAVG"]*100), columns=["density"])
air_density["Date"] = air_density.index

In [15]:
topo_polution = topo_polution.join(air_density, on = "Date",lsuffix='_caller', rsuffix='_other')

In [16]:
topo_polution["Ind_P10"] = topo_polution["Ind_P10"]/topo_polution["density"]

In [17]:
topo_polution.describe()

,Date_caller,Lat,Lon,Ind_P10,density,Date_other
count,3920.000000,3920.000000,3920.000000,3920.000000,3920.000000,3920.000000
mean,9.500000,42.679796,23.308214,20.997868,1.260852,9.500000
std,5.767017,0.037089,0.054715,80.640884,0.027713,5.767017
min,0.000000,42.620000,23.220000,0.000000,1.201294,0.000000
25%,4.750000,42.647598,23.260714,0.000000,1.246616,4.750000
50%,9.500000,42.679796,23.308214,0.000000,1.263779,9.500000
75%,14.250000,42.711994,23.355714,0.000000,1.276042,14.250000
max,19.000000,42.739592,23.396429,804.432461,1.313515,19.000000


In [18]:
def make_heatmap(df, timestamp, metric):
    """Create a Heat Map of Sofia for a given timestamp to visualize a given metric.
    For example, to visualize PM10 pollution
    
    The map is interactive.

    The map also visualizes clusters of the locations.

    Keyword arguments:
    df     -- the data frame with time, longitude, lattitude and the chosen metric
    timestamp -- the point in time for which to visualize the heat map
    metric -- the metric, used for visualization
    """
    points = df[df["Date_caller"] == int(timestamp)]
    
    folium_map = folium.Map(location=sofia_center,
                            zoom_start=11,
                            tiles='Stamen Terrain')

    marker_cluster = plugins.MarkerCluster().add_to(folium_map)
    
    for i in range(0, len(points)):
        point = points.iloc[i]

        folium.Marker(
            [point['Lat'], point['Lon']],
            popup=str(point['Ind_P10'])
        ).add_to(marker_cluster)
        
#         folium.Circle(
#             radius=10,
#             location=[point['latitude'], point['longitude']],
#             popup=str(point['P1']),
#             color='#333333',
#             fill=False
#         ).add_to(folium_map)

    plugins.MarkerCluster().add_to(folium_map)
        
    # plot heatmap
    folium_map.add_child(plugins.HeatMap(
        points[['Lat', 'Lon', metric]].as_matrix(),
        min_opacity=0.2,
        max_val=points[metric].max(),
        radius=30, blur=17,
        max_zoom=1
    ))

    # You can also save the interactive heat map as an HTML file
    # folium_map.save("output-map.html")
    
    return folium_map

In [19]:
sofia_center = [42.697708, 23.321867]
make_heatmap(topo_polution, '0', 'Ind_P10')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [22]:
df_stations

,AirQualityStationEoICode,CommonName,Longitude,Latitude
0,BG0040A,Nadezhda,23.310972,42.732292
1,BG0050A,Hipodruma,23.296786,42.680558
2,BG0052A,Druzhba,23.400164,42.666508
3,BG0073A,IAOS/Pavlovo,23.268403,42.669797


In [31]:
df_ind["t/y"] = df_ind["t/y"] * 1000000/(365*24*60*60) # convert the debit to g/s
df_meteo["sfcWindAVG"] = df_meteo["sfcWindAVG"] * 1000/3600 # convert wind speed in m/s

stations_polution = pd.DataFrame({'Date': [], 'Lat': [], 'Lon': [], 'Ind_P10': []})

for i in range(0,df_meteo.shape[0]):
    for j in range(0,df_stations.shape[0]):
        c = 0
        for k in range(0,df_ind.shape[0]):
            a = (df_stations["Latitude"][j], df_stations["Longitude"][j])
            b = (df_ind["Lat"][k],df_ind["Lon"][k])

            x = geopy.distance.distance(a, b).km #distance in kilometers

            if x < 1:
                sigma_y = 50.5 * x**(0.894)
                sigma_z = 22.8 * x**(0.675-1.3)
            else:
                sigma_y = 50.5 * x**(0.894)
                sigma_z = 55.4 * x**(0.305-34.0)

            c += concentration(y = x ,
                              q = df_ind["t/y"][k],
                              u = df_meteo["sfcWindAVG"][i],
                              h = df_ind["m"][k],
                              sigma_y = sigma_y,
                              sigma_z = sigma_z)
        stations_polution = stations_polution.append({'Date': i, 'Lat': df_stations["Latitude"][j], 'Lon': df_stations["Longitude"][j], 'Ind_P10': c}, ignore_index=True)
    print('Calculations for day {} are ready.'.format(i))

Calculations for day 0 are ready.
Calculations for day 1 are ready.
Calculations for day 2 are ready.
Calculations for day 3 are ready.
Calculations for day 4 are ready.
Calculations for day 5 are ready.
Calculations for day 6 are ready.
Calculations for day 7 are ready.
Calculations for day 8 are ready.
Calculations for day 9 are ready.
Calculations for day 10 are ready.
Calculations for day 11 are ready.
Calculations for day 12 are ready.
Calculations for day 13 are ready.
Calculations for day 14 are ready.
Calculations for day 15 are ready.
Calculations for day 16 are ready.
Calculations for day 17 are ready.
Calculations for day 18 are ready.
Calculations for day 19 are ready.


In [32]:
stations_polution = stations_polution.join(air_density, on = "Date",lsuffix='_caller', rsuffix='_other')
stations_polution["Ind_P10"] = stations_polution["Ind_P10"]/stations_polution["density"]

In [37]:
stations_polution_file = "stations_polution.csv"
stations_polution.to_csv(folder+stations_polution_file)

In [34]:
polution = topo_polution.append(stations_polution)

In [14]:
polution_file = "industrial_polution.csv"
polution.to_csv(folder+polution_file)

NameError: name 'polution' is not defined